In [0]:
adlsAccountName = "synapsetiphackathon"
sourceAdlsContainerName = "landingzone"
sinkAdlsContainerName = "stagingzone"
sourceAdlsFolderName = "raw/customer_address"
sinkAdlsFolderName = "rawdata/tiphack/customer_address"

In [0]:
spark.conf.set(
    "fs.azure.account.key." + adlsAccountName + ".dfs.core.windows.net",
    dbutils.secrets.get(scope="autoloaderphotonscope",key="Tip-adls2-secret"))

In [0]:
dbutils.fs.ls("abfss://" + sourceAdlsContainerName + "@" + adlsAccountName + ".dfs.core.windows.net/")

Out[3]: [FileInfo(path='abfss://landingzone@synapsetiphackathon.dfs.core.windows.net/raw/', name='raw/', size=0, modificationTime=1643589337000)]

In [0]:
SubscriptionID = dbutils.secrets.get("autoloaderphotonscope","SubscriptionID")
DirectoryID = dbutils.secrets.get("autoloaderphotonscope","DirectoryID")
ServicePrincipalAppID = dbutils.secrets.get("autoloaderphotonscope","ServicePrincipalAppID")
ServicePrincipalSecret = dbutils.secrets.get("autoloaderphotonscope","autoloaderphotonappsecret")
ResourceGroup = dbutils.secrets.get("autoloaderphotonscope","TipResourceGroup")
BlobConnectionKey = dbutils.secrets.get("autoloaderphotonscope","Tip-adls2-secret")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": ServicePrincipalAppID,
           "fs.azure.account.oauth2.client.secret": ServicePrincipalSecret,
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/"+DirectoryID+"/oauth2/token"}

In [0]:
dbutils.fs.unmount("/mnt/source")
dbutils.fs.unmount("/mnt/sink") 

/mnt/source has been unmounted.
/mnt/sink has been unmounted.
Out[6]: True

In [0]:
dbutils.fs.mount(
  source = "abfss://"+sourceAdlsContainerName+"@"+adlsAccountName+".dfs.core.windows.net/",
  mount_point = "/mnt/source",
  extra_configs = configs)


dbutils.fs.mount(
  source = "abfss://"+sinkAdlsContainerName+"@"+adlsAccountName+".dfs.core.windows.net/",
  mount_point = "/mnt/sink",
  extra_configs = configs)

Out[7]: True

In [0]:
dbutils.fs.ls ("/mnt/")

Out[8]: [FileInfo(path='dbfs:/mnt/dlt/', name='dlt/', size=0, modificationTime=1642004768000),
 FileInfo(path='dbfs:/mnt/sink/', name='sink/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/source/', name='source/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/training/', name='training/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/unify/', name='unify/', size=0, modificationTime=0)]

In [0]:
# Syntax from Synapse Spark
#source_files = dbutils.fs.ls ("/mnt/source/")
#for file in source_files:
    #print(file.name, file.isDir, file.isFile, file.path, file.size)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-657881968259302> in <module> 
 ----> 1 source_files = dbutils . fs . ls ( "/mnt/source/" ) 
 2 for file in source_files : 
 3 print ( file . name , file . isDir , file . isFile , file . path , file . size ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 385 exc . __context__ = None 
 386 exc . __cause__ = None 
 --> 387 raise exc
 388 
 389 return f_with_exception_handling

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: Operation failed: "This request is not authorized to perform this operation using this permission.", 403, GET, https://synapsetiphackathon.dfs.core.windows.net/landingzone?upn=false&resource=filesystem&maxResults=500&timeout=90&recursive=false, AuthorizationPermissionMismatch, "This request is not authorized to perform this operation using this permission. RequestId:06dff7e3-a01f-0048-7eae-17d3b6000000 Time:2022-02-01T20:56:34.7110326Z"
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.services.AbfsRestOperation.execute(AbfsRestOperation.java:241)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.services.AbfsClient.listPath(AbfsClient.java:315)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystemStore.listStatus(AzureBlobFileSystemStore.java:961)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystemStore.listStatus(AzureBlobFileSystemStore.java:923)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.listStatus(AzureBlobFileSystem.java:517)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:95)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:66)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:341)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:435)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:455)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:215)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:95)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:213)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:210)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:251)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:243)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:430)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:350)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperationWithResultTags(DatabricksFileSystemV2.scala:510)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:341)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:313)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:510)
	at com.databricks.backend.daemon.data.client.DBFSV2.listStatus(DatabricksFileSystemV2.scala:92)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala

##Intialize AutoLoader

In [0]:
from pyspark.sql.types import *
import json

jschema = '{"fields":[{"metadata":{},"name":"ca_address_sk","nullable":false,"type":"integer"},{"metadata":{},"name":"ca_address_id","nullable":false,"type":"string"},{"metadata":{},"name":"ca_street_number","nullable":true,"type":"string"},{"metadata":{},"name":"ca_street_name","nullable":true,"type":"string"},{"metadata":{},"name":"ca_street_type","nullable":true,"type":"string"},{"metadata":{},"name":"ca_suite_number","nullable":true,"type":"string"},{"metadata":{},"name":"ca_city","nullable":true,"type":"string"},{"metadata":{},"name":"ca_county","nullable":true,"type":"string"},{"metadata":{},"name":"ca_state","nullable":true,"type":"string"},{"metadata":{},"name":"ca_zip","nullable":true,"type":"string"},{"metadata":{},"name":"ca_country","nullable":true,"type":"string"},{"metadata":{},"name":"ca_gmt_offset","nullable":true,"type":"decimal(5,2)"},{"metadata":{},"name":"ca_location_type","nullable":true,"type":"string"}],"type":"struct"}'

schema = StructType.fromJson(json.loads(jschema))

In [0]:
cloudfile = {
  "cloudFiles.subscriptionId":SubscriptionID,
  "cloudFiles.format":"csv",
  "cloudFiles.tenantId":DirectoryID,
  "cloudFiles.clientId":ServicePrincipalAppID,
  "cloudFiles.clientSecret":ServicePrincipalSecret,
  "cloudFiles.resourceGroup":ResourceGroup,
  "cloudFiles.useNotifications": "true", 
}

##Build Streaming Dataframe

In [0]:
filePath = "/mnt/source/"+sourceAdlsFolderName+"/"
df = (spark
      .readStream
      .format("cloudFiles")
      .schema(schema)
      .options(**cloudfile)
      .option("delimiter", "|") 
      .option("Header",False)
      .option("cloudFiles.schemaLocation","/mnt/source/"+sourceAdlsFolderName+"/_schema") 
      .load(filePath))

In [0]:
from pyspark.sql.functions import input_file_name, count
filesdf = (df
          .withColumn("file",input_file_name())
          .groupBy("file")
          .agg(count("*"))
          )
display(filesdf)

file,count(1)
/mnt/source/raw/customer_address/customer_address_3_24.dat,41667
/mnt/source/raw/customer_address/customer_address_11_24.dat,41667
/mnt/source/raw/customer_address/customer_address_7_24.dat,41667
/mnt/source/raw/customer_address/customer_address_14_24.dat,41667
/mnt/source/raw/customer_address/customer_address_5_24.dat,41667
/mnt/source/raw/customer_address/customer_address_13_24.dat,41667
/mnt/source/raw/customer_address/customer_address_8_24.dat,41667
/mnt/source/raw/customer_address/customer_address_4_24.dat,41667
/mnt/source/raw/customer_address/customer_address_17_24.dat,41666
/mnt/source/raw/customer_address/customer_address_22_24.dat,41666


In [0]:
from delta.tables import *
def upsertToDelta(microBatchOutputDF, batchId):
  
  deltadf = DeltaTable.forName(spark,"saleslt.address")
  
  (deltadf.alias("t")
  .merge(
    microBatchOutputDF.alias("s"),
    "s.AddressID = t.AddressID")
  .whenMatchedUpdateAll()
  .whenNotMatchedInsertAll()
  .execute()
  )

In [0]:
streamQuery = (df.writeStream
 .format("delta")
 .outputMode("append")
 #.foreachBatch(upsertToDelta) # Comment this out first time you run
 .queryName("c-changeLoader-merge-customer-address") # Comment this out first time you run
 #.trigger(once=True)
 .option("checkpointLocation", "/mnt/sink/"+sinkAdlsFolderName+"/AutoLoader/_checkpoint")
 .start("/mnt/sink/"+sinkAdlsFolderName+"/AutoLoader/data/")
)

In [0]:
dfsql = spark.read.format("delta").load("/mnt/sink/"+sinkAdlsFolderName+"/AutoLoader/data/")
dfsql.show()

+-------------+----------------+----------------+---------------+--------------+---------------+--------------+---------------+--------+------+-------------+-------------+----------------+
ca_address_sk| ca_address_id|ca_street_number| ca_street_name|ca_street_type|ca_suite_number| ca_city| ca_county|ca_state|ca_zip| ca_country|ca_gmt_offset|ca_location_type|
+-------------+----------------+----------------+---------------+--------------+---------------+--------------+---------------+--------+------+-------------+-------------+----------------+
 375004|AAAAAAAAMNILFAAA| 648|Chestnut Walnut| Blvd| Suite 100| Prosperity| El Paso County| TX| 79089|United States| -6.00| single family|
 375005|AAAAAAAANNILFAAA| 591| 6th Lincoln| Lane| Suite 450|Pleasant Grove|Guernsey County| OH| 44136|United States| -5.00| single family|
 375006|AAAAAAAAONILFAAA| 774| Hickory 14th| Parkway| Suite 450| Oakwood| White County| IN| 40169|United States| -5.00| apartment|
 375007|AAAAAAAAPNILFAAA| 728| Green | Ct.| Suite 10| Stringtown| Cedar County| IA| 50162|United States| -6.00| condo|
 375008|AAAAAAAAAOILFAAA| 510| 13th 5th| Wy| Suite 340| Newtown|Buncombe County| NC| 21749|United States| -5.00| apartment|
 375009|AAAAAAAABOILFAAA| 303| 11th | Blvd| Suite F| Greenville| Lee County| KY| 41387|United States| -5.00| apartment|
 375010|AAAAAAAACOILFAAA| 292| 3rd | Ave| Suite 120| Walnut Grove| Jackson County| MI| 47752|United States| -5.00| single family|
 375011|AAAAAAAADOILFAAA| 576| Sunset 10th| Avenue| Suite 100| Mountain View| Lyon County| MN| 54466|United States| -6.00| single family|
 375012|AAAAAAAAEOILFAAA| 68| Hillcrest 14th| RD| Suite 0| Waterloo| Cedar County| IA| 51675|United States| -6.00| single family|
 375013|AAAAAAAAFOILFAAA| 424|Birch Jefferson| Ln| Suite 410| Lakeside| Meade County| KS| 69532|United States| -6.00| single family|
 375014|AAAAAAAAGOILFAAA| 203| Ash Sixth| Cir.| Suite 0| Walnut Grove| Kimball County| NE| 67752|United States| -7.00| single family|
 375015|AAAAAAAAHOILFAAA| 485| Poplar Sunset| Drive| Suite 450| Riverside| Clay County| TX| 79231|United States| -6.00| single family|
 375016|AAAAAAAAIOILFAAA| 200| Ash | Road| Suite X| Elkton|Costilla County| CO| 83481|United States| -7.00| condo|
 375017|AAAAAAAAJOILFAAA| 713| Lake | Ln| Suite 490| Colfax| Marion County| SC| 22565|United States| -5.00| single family|
 375018|AAAAAAAAKOILFAAA| 413| Davis | Wy| Suite 170| Edgewood| Knox County| IL| 60069|United States| -6.00| apartment|
 375019|AAAAAAAALOILFAAA| 655| Railroad Ash| Blvd| Suite H| Farmington| Phelps County| MO| 69145|United States| -6.00| single family|
 375020|AAAAAAAAMOILFAAA| 265| Main Cherry| Circle| Suite C| Brownsville| Logan County| KY| 49310|United States| -5.00| apartment|
 375021|AAAAAAAANOILFAAA| 557| Pine Oak| Pkwy| Suite 280| Elba| Harlan County| KY| 40262|United States| -6.00| apartment|
 375022|AAAAAAAAOOILFAAA| 887| Lakeview 5th| Pkwy| Suite X| Georgetown| Lynchburg city| VA| 27057|United States| -5.00| condo|
 375023|AAAAAAAAPOILFAAA| 39| Franklin | Avenue| Suite B| Spring Valley| Thomas County| KS| 66060|United States| -6.00| apartment|
+-------------+----------------+----------------+---------------+--------------+---------------+--------------+---------------+--------+------+-------------+-------------+----------------+
only showing top 20 rows

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS tip")
spark.sql("CREATE TABLE IF NOT EXISTS tip.customer_address USING DELTA LOCATION '/mnt/sink/"+sinkAdlsFolderName+"/AutoLoader/data/'")

Out[15]: DataFrame[]

In [0]:
%sql

SELECT * FROM tip.customer_address where ca_zip = 79089

ca_address_sk,ca_address_id,ca_street_number,ca_street_name,ca_street_type,ca_suite_number,ca_city,ca_county,ca_state,ca_zip,ca_country,ca_gmt_offset,ca_location_type
375004,AAAAAAAAMNILFAAA,648,Chestnut Walnut,Blvd,Suite 100,Prosperity,El Paso County,TX,79089,United States,-6.00,single family
382898,AAAAAAAACLHNFAAA,15,Sixth Church,RD,Suite 80,Prosperity,Jackson County,AR,79089,United States,-6.00,single family
384961,AAAAAAAABMPNFAAA,388,5th,Drive,Suite I,Prosperity,Zapata County,TX,79089,United States,-6.00,single family


In [0]:
%sql

SELECT count(*) FROM tip.customer_address

count(1)
1000000
